In this notebook we want to try an experiment with time prediction of the Sun. A way to pass data by sequence is defined and some changes in the architecture but still some issues.

In [41]:
import logging
import sys
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
from matplotlib.pyplot import imshow
import operator
from functools import reduce

import torch
from torch import nn, optim
from sdo.sdo_dataset import SDO_Dataset
from torch.utils.data import DataLoader
from torch.nn import functional as F

from sdo.models.encoder_decoder import AutoEncoder

%matplotlib inline

[2019-07-24 02:35:38] DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [42]:
#just a way to get nice logging messages from the sdo package
logformat = "[%(asctime)s] %(levelname)s:%(name)s:%(message)s"
logging.basicConfig(level=logging.DEBUG, stream=sys.stdout, format=logformat, datefmt="%Y-%m-%d %H:%M:%S")

In [43]:
# let's start simple by considering one single channel
# single channel has a bug in the data loader (one xextra dim is added)
subsample = 1
original_ratio = 512
img_shape = int(original_ratio/subsample)
instr = ['AIA', 'AIA']
channels = ['0171', '0131']

In [55]:
#some cuda initialization
torch.backends.cudnn.enabled = True
cuda_device = 5
if not torch.cuda.is_available():
    raise RuntimeError("CUDA not available! Unable to continue")
device = torch.device("cuda:{}".format(cuda_device))
print("Using device {} for training, current device: {}, total devices: {}".format(
device, torch.cuda.current_device(), torch.cuda.device_count()))

Using device cuda:5 for training, current device: 0, total devices: 6


In [56]:
%%time
train_data = SDO_Dataset(device=device, instr=instr, channels=channels, yr_range=[2018,2018], 
                         mnt_step=1, day_step=1, h_step=1, min_step=12, subsample=subsample, 
                         test_ratio=0.3, normalization=0, scaling=True)

[2019-07-24 02:39:43] INFO:sdo.sdo_dataset:Loading SDOML from "/gpfs/gpfs_gl4_16mb/b9p111/fdl_sw/SDOML"
[2019-07-24 02:39:43] INFO:sdo.sdo_dataset:Training on months "[1 2 3 4 5 6 7]"
[2019-07-24 02:39:43] DEBUG:sdo.sdo_dataset:Timestamps requested values: 
[2019-07-24 02:39:43] DEBUG:sdo.sdo_dataset:Years: 2018
[2019-07-24 02:39:43] DEBUG:sdo.sdo_dataset:Months: 1,2,3,4,5,6,7
[2019-07-24 02:39:43] DEBUG:sdo.sdo_dataset:Days: 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
[2019-07-24 02:39:43] DEBUG:sdo.sdo_dataset:Hours: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
[2019-07-24 02:39:43] DEBUG:sdo.sdo_dataset:Minutes: 0,12,24,36,48
[2019-07-24 02:39:43] INFO:sdo.sdo_dataset:Max number of timestamps: 26040
[2019-07-24 02:39:45] INFO:sdo.sdo_dataset:Timestamps found in the inventory: 24503 (0.94)
[2019-07-24 02:39:49] INFO:sdo.sdo_dataset:N timestamps discarded because channel is missing = 20 (0.00082)
[2019-07-24 02:39:49] INFO:sdo.s

In [96]:
"""
In this module we want to define an encoder/decoder RNN architecture
"""
def prod(iterable):
    return reduce(operator.mul, iterable, 1)


class EncoderDecoderRNN(nn.Module):
    """
    This is a RNN that output the next image of a sequence of images.
    """
    def __init__(self, input_shape=[1, 512, 512], lstm_input_dim=1024):
        super(EncoderDecoderRNN, self).__init__()
        self.lstm_input_dim = lstm_input_dim
        self.num_channels = input_shape[0]
        self.input_shape = input_shape
        self.conv1 = nn.Conv2d(in_channels=self.num_channels, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3)
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3)
        
        self.dconv5 = nn.ConvTranspose2d(in_channels=64, out_channels=128, kernel_size=3)
        self.dconv4 = nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=3)
        self.dconv3 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3)
        self.dconv2 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3)
        self.dconv1 = nn.ConvTranspose2d(in_channels=32, out_channels=self.num_channels, kernel_size=3)

        self.pool = nn.MaxPool2d(2, 2, return_indices=True)
        self.unpool = nn.MaxUnpool2d(2, 2)
      
        sample_encoder_input = torch.zeros(input_shape)
        sample_encoder_output = self._encoder(sample_encoder_input.unsqueeze(0))[0]
        sample_encoder_output_dim = sample_encoder_output.nelement()
        print("sample_encoder_output_dim",sample_encoder_output_dim)

        # TODO: Choose a better hidden_size.
        # TODO: Choose more stacked num_layers.
        self.lin1 = nn.Linear(sample_encoder_output_dim,  self.lstm_input_dim)
        self.lstm = nn.LSTM(input_size=self.lstm_input_dim, hidden_size=self.lstm_input_dim, 
                            num_layers=1, batch_first=True)
        self.lstm_fc = nn.Linear(in_features=self.lstm_input_dim, out_features=sample_encoder_output_dim)

        print('Autoencoder architecture:')
        print('Input shape: {}'.format(input_shape))
        print('Input dim  : {}'.format(prod(input_shape)))
        print('Encoded dim: {}'.format(sample_encoder_output_dim))
        print('Learnable params: {}'.format(sum([p.numel() for p in self.parameters()])))

    def _encoder(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x, indices1 = self.pool(x)
        x = self.conv3(x)
        x, indices2 = self.pool(x)
        x = self.conv4(x)
        x, indices3 = self.pool(x)
        x = self.conv5(x)
        x, indices4 = self.pool(x)
        x = F.relu(x)
        return x, indices1, indices2, indices3, indices4

    def _decoder(self, x, indices1, indices2, indices3, indices4):
        # somthing is wrong here
        x = x.view(64, 30, 30)
        x = self.unpool(x, indices4)
        x = self.dconv5(x)
        x = F.relu(x)
        x = self.unpool(x, indices3)
        x = self.dconv4(x)
        x = F.relu(x)
        x = self.unpool(x, indices2)
        x = self.dconv3(x)
        x = F.relu(x)
        x = self.unpool(x, indices1)
        x = self.dconv2(x)
        x = F.relu(x)
        x = self.dconv1(x)
        x = torch.relu(x)
        return x

    def forward(self, x):
        # x has shape B x T x C x H x W
        #import pdb
        #pdb.set_trace()
        batch_size = x.shape[0]
        sequence_size = x.shape[1]
        encoded_x = []
        decoded_x = []
        indices_t = []
        for t in range(sequence_size):
            xt = x[:, t, :, :, :]
            xt, indices1, indices2, indices3, indices4 = self._encoder(xt)
            print("Encoded time stamp", xt.shape)
            Shap = xt.shape
            xt = xt.view(Shap[0], -1)
            xt = self.lin1(xt)
            xt = F.relu(xt)
            print("Encoded time stamp post FC", xt.shape)
            encoded_x.append(xt)
            indices_t.append([indices1, indices2, indices3, indices4])
            #xt = xt.view(Shap[0], 1, -1)
        x = torch.stack(encoded_x)
        x, _ = self.lstm(x)
        print("Output LSTM", x.shape)
        x = self.lstm_fc(x)
        print("Input decoder", x.shape)
        for t in range(sequence_size):
            xt = x[t, :, :]
            xt = self._decoder(xt, *indices_t[t])
            decoded_x.append(xt)
        x = torch.stack(decoded_x)
        return x

In [97]:
#let's define a RNN model
model = EncoderDecoderRNN(input_shape=[2, 512, 512])
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_function = nn.MSELoss()

sample_encoder_output_dim 57600
Autoencoder architecture:
Input shape: [2, 512, 512]
Input dim  : 524288
Encoded dim: 57600
Learnable params: 127048834


In [98]:
# is the shuffling in the data loader shuffling the order of the batches or inside the batch
seq_len = 6
train_data_loader = DataLoader(train_data, batch_size=seq_len, shuffle=False)

In [99]:
# training loop
import pdb
model.cuda(cuda_device)
len_data = train_data.__len__()
log_interval = 1
# for now I am training on a single year
n_epochs = 4
train_loss = []
for epoch in range(n_epochs):
    for batch_index, batch in enumerate(train_data_loader):
        # reshaping in order to get sequence
        # batch_size define the length of the sequence
        print("Batch", batch_train.shape)
        batch = batch.to(cuda_device)
        batch = batch.unsqueeze(0)
        truth = batch[:,-1,:, :, :]
        print("Truth size", truth.shape)
        batch_train = batch[:, :-1, :, :, :]
        print("Batch train", batch_train.shape)
        truth = truth.unsqueeze(1) 
        optimizer.zero_grad()
        output = model(batch_train)
        loss = distance(output, truth)
        train_loss.append(float(loss))
        loss.backward()
        optimizer.step()
        if batch_index % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_index * len(data), len_data, 
                100.*(batch_index* len(data)) / len_data, 
                loss.item() / len(data)))

Batch torch.Size([1, 5, 2, 512, 512])
Truth size torch.Size([1, 2, 512, 512])
Batch train torch.Size([1, 5, 2, 512, 512])
Encoded time stamp torch.Size([1, 64, 30, 30])
Encoded time stamp post FC torch.Size([1, 1024])
Encoded time stamp torch.Size([1, 64, 30, 30])
Encoded time stamp post FC torch.Size([1, 1024])
Encoded time stamp torch.Size([1, 64, 30, 30])
Encoded time stamp post FC torch.Size([1, 1024])
Encoded time stamp torch.Size([1, 64, 30, 30])
Encoded time stamp post FC torch.Size([1, 1024])
Encoded time stamp torch.Size([1, 64, 30, 30])
Encoded time stamp post FC torch.Size([1, 1024])
Output LSTM torch.Size([5, 1, 1024])
Input decoder torch.Size([5, 1, 57600])


IndexError: tuple index out of range